## Testing

In [ ]:
from se4g_dir import set_the_working_directory
from se4g_helper import *

### Data Management

In [ ]:
def test_connect_right_now(self):
    conn = connect_right_now()
    self.assertIsNotNone(conn)


In [ ]:
def test_connect_with_sqlalchemy(self):
    engine = connect_with_sqlalchemy()
    self.assertIsNotNone(engine)


In [ ]:
def test_insert_data(self):
    conn = connect_right_now()
    columns = ['column1', 'column2']  # Replace with your column names
    rows = [(1, 'data1'), (2, 'data2')]  # Replace with your sample rows
    insert_data('test_table', rows, conn, columns)

    # Now, fetch the inserted data and verify its correctness
    cur = conn.cursor()
    cur.execute("SELECT * FROM test_table")
    inserted_data = cur.fetchall()

    self.assertEqual(len(inserted_data), len(rows))
    self.assertEqual(inserted_data, rows)

    cur.close()
    conn.close()


In [ ]:
def test_table_exists(self):
    conn = connect_right_now()
    cur = conn.cursor()

    # Create a temporary table
    cur.execute("CREATE TABLE temp_table (id SERIAL PRIMARY KEY, name VARCHAR)")

    # Check if the table exists
    exists = table_exists('temp_table', conn)
    self.assertTrue(exists)

    # Drop the temporary table
    cur.execute("DROP TABLE temp_table")

    cur.close()
    conn.close()


In [ ]:
def test_update_DB(self):
    conn = connect_right_now()
    cur = conn.cursor()

    # Create a temporary table
    cur.execute("CREATE TABLE temp_table (id SERIAL PRIMARY KEY, name VARCHAR)")

    # Insert some initial data into the table
    initial_data = [(1, 'data1'), (2, 'data2')]
    insert_data('temp_table', initial_data, conn, ['id', 'name'])

    # Define the new rows to be inserted
    new_rows = [(3, 'data3'), (4, 'data4')]

    # Call the update_DB() function
    updated_rows = update_DB(new_rows, conn, 'temp_table', ['id', 'name'])

    # Fetch the updated data
    cur.execute("SELECT * FROM temp_table")
    updated_data = cur.fetchall()

    # Verify the correctness of the updated data
    expected_data = initial_data + updated_rows
    self.assertEqual(len(updated_data), len(expected_data))
    self.assertEqual(updated_data, expected_data)

    # Drop the temporary table
    cur.execute("DROP TABLE temp_table")

    cur.close()
    conn.close()


In [ ]:
def test_update_DB_from_CSV(self):
    conn = connect_right_now()
    cur = conn.cursor()

    # Create a temporary table
    cur.execute("CREATE TABLE temp_table (id SERIAL PRIMARY KEY, name VARCHAR)")

    # Insert some initial data into the table
    initial_data = [(1, 'data1'), (2, 'data2')]
    insert_data('temp_table', initial_data, conn, ['id', 'name'])

    # Create a new DataFrame representing the data from the CSV file
    new_data = pd.DataFrame([(3, 'data3'), (4, 'data4')], columns=['id', 'name'])

    # Call the update_DB_from_CSV() function
    updated_data = update_DB_from_CSV(new_data, conn, 'temp_table', ['id', 'name'])

    # Fetch the updated data
    cur.execute("SELECT * FROM temp_table")
    updated_table = cur.fetchall()

    # Verify the correctness of the updated data
    expected_table = initial_data + updated_data
    self.assertEqual(len(updated_table), len(expected_table))
    self.assertEqual(updated_table, expected_table)

    # Drop the temporary table
    cur.execute("DROP TABLE temp_table")

    cur.close()
    conn.close()


In [ ]:
def test_update_dashboard_DB_from_CSV(self):
    conn = connect_right_now()
    engine = connect_with_sqlalchemy()

    # Create a temporary table for the dashboard
    cur = conn.cursor()
    cur.execute("CREATE TABLE temp_dashboard (id SERIAL PRIMARY KEY, name VARCHAR)")

    # Insert some initial data into the dashboard table
    initial_data = [(1, 'data1'), (2, 'data2')]
    insert_data('temp_dashboard', initial_data, conn, ['id', 'name'])

    # Create a new DataFrame representing the data from the CSV file
    new_data = pd.DataFrame([(3, 'data3'), (4, 'data4')], columns=['id', 'name'])

    # Call the update_dashboard_DB_from_CSV() function
    updated_data = update_dashboard_DB_from_CSV(new_data, conn, engine, 'temp_dashboard', ['id', 'name'])

    # Fetch the updated data from the dashboard table
    cur.execute("SELECT * FROM temp_dashboard")
    updated_table = cur.fetchall()

    # Verify the correctness of the updated data
    expected_table = initial_data + updated_data
    self.assertEqual(len(updated_table), len(expected_table))
    self.assertEqual(updated_table, expected_table)

    # Drop the temporary table
    cur.execute("DROP TABLE temp_dashboard")

    cur.close()
    conn.close()


In [ ]:
# Test download_request function
def test_download_request():
    COUNTRIES = ['AD', 'SE']
    POLLUTANTS = ['PM10', 'NO2']
    folder_out = 'test_data'
    dir = download_request(COUNTRIES, POLLUTANTS, folder_out)
    
    assert os.path.exists(os.path.join(folder_out, dir))
    assert len(os.listdir(os.path.join(folder_out, dir))) == len(COUNTRIES) * len(POLLUTANTS)

# Test build_dataframe function
def test_build_dataframe():
    dir = 'test_dir'
    COUNTRIES = ['AD', 'SE']
    POLLUTANTS = ['PM10', 'NO2']
    folder_out = 'test_data'
    df_columns = ['pollutant', 'value_datetime_begin', 'value_numeric']
    df = build_dataframe(dir, COUNTRIES, POLLUTANTS, folder_out, df_columns)
    
    assert isinstance(df, pd.DataFrame)
    assert set(df.columns) == set(df_columns)
    assert not df.empty

# Test update_dataset function
def test_update_dataset():
    folder_out = 'test_data'
    fileName = 'test_dataset.csv'
    
    # Create a dummy existing dataset
    existing_data = pd.DataFrame({
        'pollutant': ['PM10', 'PM2.5'],
        'value_datetime_begin': ['2022-01-01 00:00:00', '2022-01-02 00:00:00'],
        'value_numeric': [20, 30]
    })
    existing_data.to_csv(os.path.join(folder_out, fileName), index=False)
    
    # Create a new DataFrame with additional rows
    new_data = pd.DataFrame({
        'pollutant': ['NO2', 'NO2'],
        'value_datetime_begin': ['2022-01-03 00:00:00', '2022-01-04 00:00:00'],
        'value_numeric': [40, 50]
    })
    update_dataset(new_data, folder_out, fileName)
    
    # Check if the dataset is updated correctly
    updated_data = pd.read_csv(os.path.join(folder_out, fileName))
    assert len(updated_data) == len(existing_data) + len(new_data)
    assert updated_data['value_datetime_begin'].max() == new_data['value_datetime_begin'].max()

# Test update_dashboard_dataset function
def test_update_dashboard_dataset():
    folder_out = 'test_data'
    fileName = 'test_dashboard_dataset.csv'
    
    # Create a dummy existing dataset
    existing_data = pd.DataFrame({
        'pollutant': ['PM10', 'PM2.5'],
        'network_countrycode': ['AD', 'SE'],
        'value_datetime_end': ['2022-01-01 00:00:00+00:00', '2022-01-02 00:00:00+00:00'],
        'value_numeric': [20, 30]
    })
    existing_data.to_csv(os.path.join(folder_out, fileName), index=False)
    
    # Create a new DataFrame with additional rows
    new_data = pd.DataFrame({
        'pollutant': ['NO2', 'NO2'],
        'network_countrycode': ['AD', 'SE'],
        'value_datetime_end': ['2022-01-03 00:00:00+00:00', '2022-01-04 00:00:00+00:00'],
        'value_numeric': [40, 50]
    })
    update_dashboard_dataset(new_data, folder_out)
    
    # Check if the dataset is updated correctly
    updated_data = pd.read_csv(os.path.join(folder_out, fileName))
    assert len(updated_data) == len(existing_data) + len(new_data)
    assert updated_data['value_datetime_end'].max() == new_data['value_datetime_end'].max()

# Test update_data function
def test_update_data():
    # Create a dummy directory and files
    dir = 'test_dir'
    COUNTRIES = ['AD', 'SE']
    POLLUTANTS = ['PM10', 'NO2']
    folder_out = 'test_data'
    os.makedirs(os.path.join(folder_out, dir), exist_ok=True)
    for country in COUNTRIES:
        for pollutant in POLLUTANTS:
            fileName = "%s_%s.csv" % (country, pollutant)
            file_path = os.path.join(folder_out, dir, fileName)
            df = pd.DataFrame({
                'pollutant': [pollutant],
                'value_datetime_begin': [datetime.now().strftime("%Y-%m-%d %H:%M:%S")],
                'value_numeric': [100]
            })
            df.to_csv(file_path, index=False)
    
    # Execute the update process
    update_data()
    
    # Check if the datasets are updated correctly
    dataset_file = os.path.join(folder_out, 'se4g_pollution_dataset.csv')
    dashboard_file = os.path.join(folder_out, 'se4g_dashboard_dataset.csv')
    assert os.path.isfile(dataset_file)
    assert os.path.isfile(dashboard_file)
    dataset = pd.read_csv(dataset_file)
    dashboard = pd.read_csv(dashboard_file)
    assert len(dataset) == len(COUNTRIES) * len(POLLUTANTS)
    assert len(dashboard) == len(COUNTRIES) * len(POLLUTANTS)

# Run the test cases
test_download_request()
test_build_dataframe()
test_update_dataset()
test_update_dashboard_dataset()
test_update_data()

### User Login & Registration

In [ ]:
import unittest
from unittest.mock import patch
from io import StringIO
from register_login import Register, Login

class TestRegisterLogin(unittest.TestCase):

    def test_new_user(self):
        register = Register()

        # Patching input() to simulate user input
        with patch('builtins.input', side_effect=['JohnDoe', 'password123']):
            register._new_user()

        # Assert that the user is added to the register_list
        self.assertEqual(register.register_list['JohnDoe'], 'password123')

    def test_remove_registration(self):
        register = Register()
        register.register_list = {'JohnDoe': 'password123'}

        # Remove an existing registration
        register.remove_registration('JohnDoe', 'password123')
        self.assertNotIn('JohnDoe', register.register_list)

        # Attempt to remove a non-existent registration
        register.remove_registration('JaneDoe', 'password456')
        self.assertNotIn('JaneDoe', register.register_list)

    def test_review_registrations(self):
        register = Register()
        register.register_list = {'JohnDoe': 'password123', 'JaneDoe': 'password456'}

        # Patching the print() function to capture the output
        with patch('sys.stdout', new=StringIO()) as fake_output:
            register._review_registrations()
            output = fake_output.getvalue()

        # Assert that the review of registrations is displayed correctly
        self.assertIn('JohnDoe', output)
        self.assertIn('JaneDoe', output)

    def test_check_credentials(self):
        login = Login()
        login.user_list = {'JohnDoe': 'password123'}

        # Check with correct credentials
        self.assertTrue(login.check_credentials('JohnDoe', 'password123'))

        # Check with incorrect password
        self.assertFalse(login.check_credentials('JohnDoe', 'password456'))

        # Check with non-existent user
        self.assertFalse(login.check_credentials('JaneDoe', 'password123'))

    # Write test cases for the remaining functions in the Login class...

if __name__ == '__main__':
    unittest.main()


### Dashboard Creation & Visualization

In [ ]:
import unittest
from unittest.mock import patch, MagicMock
from io import StringIO
import pandas as pd
import matplotlib.pyplot as plt
from descriptive_stats import DescriptiveStats

class TestDescriptiveStats(unittest.TestCase):

    @patch('descriptive_stats.connect_right_now')
    def test_create_df_from_table(self, mock_connect_right_now):
        # Mock the database connection and cursor
        mock_cursor = MagicMock()
        mock_conn = MagicMock()
        mock_conn.cursor.return_value = mock_cursor

        # Mock the execute and fetch methods
        mock_cursor.fetchall.return_value = [('A', 1), ('B', 2)]
        mock_cursor.description = [('col1',), ('col2',)]

        # Instantiate the DescriptiveStats class
        descriptive_stats = DescriptiveStats()

        # Mock the create_df_from_table method and assert the result
        with patch.object(descriptive_stats, 'create_df_from_table') as mock_create_df:
            mock_create_df.return_value = pd.DataFrame([('A', 1), ('B', 2)], columns=['col1', 'col2'])
            df = descriptive_stats.create_df_from_table('table_name')

        # Assert that the mock methods were called correctly
        mock_connect_right_now.assert_called_once()
        mock_conn.cursor.assert_called_once()
        mock_cursor.execute.assert_called_once_with('SELECT * FROM table_name;')
        mock_cursor.fetchall.assert_called_once()
        mock_cursor.close.assert_called_once()
        mock_conn.close.assert_called_once()

        # Assert the DataFrame result
        expected_df = pd.DataFrame([('A', 1), ('B', 2)], columns=['col1', 'col2'])
        pd.testing.assert_frame_equal(df, expected_df)

    def test_update_pollutants(self):
        descriptive_stats = DescriptiveStats()
        descriptive_stats.df_pollutant = pd.DataFrame({
            'station_code': ['A', 'B', 'C'],
            'pollutant': ['P1', 'P2', 'P1']
        })
        descriptive_stats.df_station = pd.DataFrame({
            'station_code': ['A', 'B', 'C'],
            'network_countrycode': ['Country1', 'Country2', 'Country1']
        })

        descriptive_stats.country_dropdown.value = 'Country1'
        descriptive_stats.update_pollutants(None)

        self.assertEqual(descriptive_stats.pollutant_dropdown.options, ['P1'])

        descriptive_stats.country_dropdown.value = 'Country2'
        descriptive_stats.update_pollutants(None)

        self.assertEqual(descriptive_stats.pollutant_dropdown.options, ['P2'])

    @patch('sys.stdout', new_callable=StringIO)
    def test_update_statistics(self, mock_stdout):
        descriptive_stats = DescriptiveStats()
        descriptive_stats.df_pollutant = pd.DataFrame({
            'station_code': ['A', 'B', 'C'],
            'pollutant': ['P1', 'P2', 'P1'],
            'value_numeric': [10, 20, 30]
        })
        descriptive_stats.df_station = pd.DataFrame({
            'station_code': ['A', 'B', 'C'],
            'network_countrycode': ['Country1', 'Country2', 'Country1']
        }) 

        descriptive_stats.country_dropdown.value = 'Country1'
        descriptive_stats.pollutant_dropdown.value = 'P1'
        descriptive_stats.update_statistics(None)

        output = mock_stdout.getvalue().strip()
        expected_output = "Mean: 20.00\nMax: 30\nMin: 10"
        self.assertEqual(output, expected_output)

    # Write test cases for the remaining functions in the DescriptiveStats class...

if __name__ == '__main__':
    unittest.main()


In [ ]:
# Test download_request function
def test_download_request():
    COUNTRIES = ['AD', 'SE']
    POLLUTANTS = ['PM10', 'NO2']
    folder_out = 'test_data'
    dir = download_request(COUNTRIES, POLLUTANTS, folder_out)
    
    assert os.path.exists(os.path.join(folder_out, dir))
    assert len(os.listdir(os.path.join(folder_out, dir))) == len(COUNTRIES) * len(POLLUTANTS)

# Test build_dataframe function
def test_build_dataframe():
    dir = 'test_dir'
    COUNTRIES = ['AD', 'SE']
    POLLUTANTS = ['PM10', 'NO2']
    folder_out = 'test_data'
    df_columns = ['pollutant', 'value_datetime_begin', 'value_numeric']
    df = build_dataframe(dir, COUNTRIES, POLLUTANTS, folder_out, df_columns)
    
    assert isinstance(df, pd.DataFrame)
    assert set(df.columns) == set(df_columns)
    assert not df.empty

# Test update_dataset function
def test_update_dataset():
    folder_out = 'test_data'
    fileName = 'test_dataset.csv'
    
    # Create a dummy existing dataset
    existing_data = pd.DataFrame({
        'pollutant': ['PM10', 'PM2.5'],
        'value_datetime_begin': ['2022-01-01 00:00:00', '2022-01-02 00:00:00'],
        'value_numeric': [20, 30]
    })
    existing_data.to_csv(os.path.join(folder_out, fileName), index=False)
    
    # Create a new DataFrame with additional rows
    new_data = pd.DataFrame({
        'pollutant': ['NO2', 'NO2'],
        'value_datetime_begin': ['2022-01-03 00:00:00', '2022-01-04 00:00:00'],
        'value_numeric': [40, 50]
    })
    update_dataset(new_data, folder_out, fileName)
    
    # Check if the dataset is updated correctly
    updated_data = pd.read_csv(os.path.join(folder_out, fileName))
    assert len(updated_data) == len(existing_data) + len(new_data)
    assert updated_data['value_datetime_begin'].max() == new_data['value_datetime_begin'].max()

# Test update_dashboard_dataset function
def test_update_dashboard_dataset():
    folder_out = 'test_data'
    fileName = 'test_dashboard_dataset.csv'
    
    # Create a dummy existing dataset
    existing_data = pd.DataFrame({
        'pollutant': ['PM10', 'PM2.5'],
        'network_countrycode': ['AD', 'SE'],
        'value_datetime_end': ['2022-01-01 00:00:00+00:00', '2022-01-02 00:00:00+00:00'],
        'value_numeric': [20, 30]
    })
    existing_data.to_csv(os.path.join(folder_out, fileName), index=False)
    
    # Create a new DataFrame with additional rows
    new_data = pd.DataFrame({
        'pollutant': ['NO2', 'NO2'],
        'network_countrycode': ['AD', 'SE'],
        'value_datetime_end': ['2022-01-03 00:00:00+00:00', '2022-01-04 00:00:00+00:00'],
        'value_numeric': [40, 50]
    })
    update_dashboard_dataset(new_data, folder_out)
    
    # Check if the dataset is updated correctly
    updated_data = pd.read_csv(os.path.join(folder_out, fileName))
    assert len(updated_data) == len(existing_data) + len(new_data)
    assert updated_data['value_datetime_end'].max() == new_data['value_datetime_end'].max()

# Test update_data function
def test_update_data():
    # Create a dummy directory and files
    dir = 'test_dir'
    COUNTRIES = ['AD', 'SE']
    POLLUTANTS = ['PM10', 'NO2']
    folder_out = 'test_data'
    os.makedirs(os.path.join(folder_out, dir), exist_ok=True)
    for country in COUNTRIES:
        for pollutant in POLLUTANTS:
            fileName = "%s_%s.csv" % (country, pollutant)
            file_path = os.path.join(folder_out, dir, fileName)
            df = pd.DataFrame({
                'pollutant': [pollutant],
                'value_datetime_begin': [datetime.now().strftime("%Y-%m-%d %H:%M:%S")],
                'value_numeric': [100]
            })
            df.to_csv(file_path, index=False)
    
    # Execute the update process
    update_data()
    
    # Check if the datasets are updated correctly
    dataset_file = os.path.join(folder_out, 'se4g_pollution_dataset.csv')
    dashboard_file = os.path.join(folder_out, 'se4g_dashboard_dataset.csv')
    assert os.path.isfile(dataset_file)
    assert os.path.isfile(dashboard_file)
    dataset = pd.read_csv(dataset_file)
    dashboard = pd.read_csv(dashboard_file)
    assert len(dataset) == len(COUNTRIES) * len(POLLUTANTS)
    assert len(dashboard) == len(COUNTRIES) * len(POLLUTANTS)

# Run the test cases
test_download_request()
test_build_dataframe()
test_update_dataset()
test_update_dashboard_dataset()
test_update_data()

In [ ]:
import unittest
from unittest.mock import patch
from dashboard import Dashboard

class DashboardTestCase(unittest.TestCase):
    @classmethod
    def setUpClass(cls):
        # Set up the test data and initialize the Dashboard instance
        cls.table_name = 'test_table'
        cls.dashboard = Dashboard(table_name=cls.table_name)

    def test_load_data(self):
        # Mock the connect_right_now function and the cursor object
        with patch('dashboard.connect_right_now') as mock_connect:
            mock_cursor = mock_connect.return_value.cursor.return_value

            # Set up the mock cursor to return test data
            test_data = [
                (1, '20210601', 'SO2', 'Country A', 10),
                (2, '20210601', 'CO', 'Country B', 5),
                (3, '20210602', 'SO2', 'Country C', 8),
            ]
            mock_cursor.fetchall.return_value = test_data
            mock_cursor.description = [('id',), ('month_day',), ('pollutant',), ('country',), ('value_numeric_mean',)]

            # Call the load_data method
            self.dashboard.load_data()

            # Verify that the data was loaded correctly into the DataFrame
            expected_columns = ['id', 'month_day', 'pollutant', 'country', 'value_numeric_mean', 'time_series', 'month_day_date']
            self.assertListEqual(list(self.dashboard.df.columns), expected_columns)
            self.assertEqual(len(self.dashboard.df), len(test_data))
            self.assertEqual(self.dashboard.df['pollutant'].nunique(), 2)
            self.assertEqual(self.dashboard.df['country'].nunique(), 3)

    def test_create_dashboard(self):
        # Call the create_dashboard method
        self.dashboard.create_dashboard()

        # Verify that the layout was created successfully
        self.assertIsNotNone(self.dashboard.app.layout)
        # Add additional assertions as needed

if __name__ == '__main__':
    unittest.main()


In [ ]:
import unittest
from unittest.mock import patch, MagicMock
from interactive import Interactive

class InteractiveTestCase(unittest.TestCase):
    def setUp(self):
        # Create mock DataFrames for pollutants and stations
        self.df_pollutant = MagicMock()
        self.df_station = MagicMock()

        # Initialize the Interactive instance with mock DataFrames
        self.interactive = Interactive(df_pollutant=self.df_pollutant, df_station=self.df_station)

    @patch('interactive.widgets')
    def test_select_pollutant(self, mock_widgets):
        # Mock the Dropdown widget and its display method
        mock_dropdown = mock_widgets.Dropdown.return_value
        mock_dropdown.options = ['SO2', 'NO2']
        mock_display = mock_widgets.display

        # Call the select_pollutant method
        result = self.interactive.select_pollutant()

        # Verify that the Dropdown widget was created and displayed
        mock_widgets.Dropdown.assert_called_with(
            options=['SO2', 'NO2'],
            description='Select pollutant:',
            layout=mock_widgets.Layout(width='250px'),
            style={'description_width': 'initial', 'min-width': '250px', 'font-size': '10pt'}
        )
        mock_display.assert_called_with(mock_dropdown)

        # Verify that the returned result is the Dropdown widget
        self.assertEqual(result, mock_dropdown)

    @patch('interactive.widgets')
    def test_select_date(self, mock_widgets):
        # Mock the Dropdown widget and its display method
        mock_dropdown = mock_widgets.Dropdown.return_value
        mock_dropdown.options = ['2023-01-01', '2023-01-02']
        mock_display = mock_widgets.display

        # Call the select_date method
        result = self.interactive.select_date()

        # Verify that the Dropdown widget was created and displayed
        mock_widgets.Dropdown.assert_called_with(
            options=['2023-01-01', '2023-01-02'],
            description='Select date:',
            layout=mock_widgets.Layout(width='250px'),
            style={'description_width': 'initial', 'min-width': '250px', 'font-size': '10pt'}
        )
        mock_display.assert_called_with(mock_dropdown)

        # Verify that the returned result is the Dropdown widget
        self.assertEqual(result, mock_dropdown)

    @patch('interactive.figure')
    @patch('interactive.output_notebook')
    @patch('interactive.show')
    def test_create_bokeh_plot(self, mock_show, mock_output_notebook, mock_figure):
        # Mock the DataFrame and Bokeh objects
        df_selected = MagicMock()
        df_filtered = MagicMock()
        source = MagicMock()
        p = mock_figure.return_value

        # Set up the mock objects and their behaviors
        self.df_pollutant.__getitem__.return_value = df_selected
        self.df_station.merge.return_value = df_filtered
        mock_figure.return_value = p
        p.line.return_value = None

        # Call the create_bokeh_plot method
        self.interactive.create_bokeh_plot(selected_pollutant='SO2', selected_date='2023-01-01')

        # Verify the expected interactions
        self.df_pollutant.__getitem__.assert_called_with(
            (self.df_pollutant['pollutant'] == 'SO2') &
            (self.df_pollutant['value_datetime_begin'].dt.date == '2023-01-01')
        )
        self.df_station.merge.assert_called_with(self.df_selected, on='station_code')
        p.line.assert_called_with(x='value_datetime_begin', y='value_numeric', source=source, line_color=color, legend_label=country)
        mock_output_notebook.assert_called_once()
        mock_show.assert_called_with(p)

if __name__ == '__main__':
    unittest.main()
